# 네이버 OpenApi를 이용한 네이버뉴스 크롤링

In [ ]:
# 필요한 라이브러리 불러오기

import numpy as np    # 행렬 연산을 위한
import requests       # HTTP 호출
import nltk           # 자연어처리 라이브러리
from konlpy.tag import Okt; okt = Okt() # 한국어 자연어 처리(품사분석)
import matplotlib.pyplot as plt # 그래프
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS  # 워드크라우드
from PIL import Image    # 이미지 처리

In [ ]:
# 네이버 뉴스 검색 api를 사용하여 최신기사 크롤링
# 검색위치,출력 건수 정해져 있어 과거의 기사를 가져올수 없는 단점

# 네이버 애플리케이션의 client_id와 client_secret 키 설정
headers = {
    'X-Naver-Client-Id' : '자신의 Client ID 입력',
    'X-Naver-Client-Secret' : '자신의 Client Secret 입력'
}

# 뉴스 검색 요청 파라미터 설정
query = '대한항공'    # 검색어
display = 1           # 검색 결과 출력 건수(10~100)
params = {
    'query' : query,
    'display' : display,    
    'start' : 1,       # 검색 시작 위치(1~1000)
    'sort' : 'sim',    # sim(유사도순), date(날짜순)
}

# 뉴스 검색 URL과 요청 파라미터
naver_news_url = 'https://openapi.naver.com/v1/search/news.json'

# 뉴스 검색 요청
res = requests.get(naver_news_url, headers=headers, params=params)
if res.status_code == 200:
    # 뉴스 검색 결과 확인
    news = res.json().get('items')
    #print(news)

In [ ]:
# 기사별로 필요한 부분 가져오기
# 네이버개발자 Documnet를 참고하여 필요한 출력결과의 필드값을 알아야함

gather = []    # 필요한 내용을 담을 리스트
for i in range(display):
    cont = news[i].get('title') + news[i].get('description')    # 제목, 내용요약만 가져옴
    cont = cont.replace('<b>', '').replace('</b>', '').replace('...','')    # HTML태그 제거
    gather.append(cont)
#print(gather)    # 결과 확인

# 크롤링한 데이터 자연어 처리

In [ ]:
# 크롤링한 기사들을 Okt를 사용하여 품사분석하고 필요없는 품사 제거

results = []    # 최종 결과물을 담을 리스트
for line in gather:
    intm = okt.pos(line, norm=True, stem=True)    # Okt를 사용해 한국어 형태소 분석(norm:정규화, stem:어간추출)
    
    r = []    # 필요없는 품사를 제외한 형태소들을 담을 리스트
    for word in intm:
        if not word[1] in ['Puctuation', 'Josa', 'Foreign', 'Number', 'Verb']:    # 개인적으로 판단하여 제거하자
            r.append(word[0])                                            # ('단어','품사')형태라 word[0],word[1]
    
    r1 = (' '.join(r)).strip()    # 필요한 형태소들만 다시 합쳐 line화(문장) 만듬
    results.append(r1)

#print(results)    # 확인용 출력  

In [ ]:
# 리스트에 담겨있는 문장들을 하나의 텍스트로 만듬

collect_text = ''

for each_line in results:
    collect_text = collect_text + each_line + '\n'
    
#print(collect_text) # 확인용 출력

In [ ]:
# 하나의 텍스트를 형태소별로 나누고 불용어를 지정하여 걸러내기

tokens_ko = okt.morphs(collect_text)    # 형태소별로 나누기

stop_words = ['.',',','\n','…','(',')','..','개','변','약','...',"'",'"']    # 불용어 지정하기

tokens_ko = [each_word for each_word in tokens_ko
            if each_word not in stop_words]         # 불용어 걸러내기

#tokens_ko     # 결과물 확인하면서 불용어 추가할것

# 자연어 처리를 통한 시각화

In [ ]:
# 자연어의 데이터 탐색을 쉽게해주는 nltk.Text
# nltk.Text를 이용해 그래프를 그리거나 워드크라우드를 만들 수 있음

ko = nltk.Text(tokens_ko, name='네이버기사')
ko.vocab().most_common(10)    # 빈도수가 가장 많은 순으로 정리(개수)

In [ ]:
# Matplotlib 그래프에 한글이 문제없이 표기되도록 함

import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unkown system... sorry~~~~')

In [ ]:
# 빈도수를 y축으로 하는 그래프를 그림

plt.figure(figsize=(15,6))
ko.plot(10)                # (보여줄 단어 개수)
plt.show()

# 이미지를 이용해 워드크라우드 만들기

In [ ]:
# 단어의 빈도수를 보여주는 워드크라우드 만들기

data = ko.vocab().most_common(50)    # 빈도수 순으로 최대 50개의 단어를 추려냄

# 딕셔너리화 시킨 후 워드크라우드 만들기
wordcloud = WordCloud(font_path=path,
                     relative_scaling=0.2,
                     background_color='white').generate_from_frequencies(dict(data))

# 워드크라우드를 그리기(Matplotlib)
plt.figure(figsize=(16,8))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
# 워드크라우드를 넣을 이미지 가져오기

mask = np.array(Image.open('./data/koreanair_image.jpg'))

# 마스킹할 이미지의 색상을 반영
from wordcloud import ImageColorGenerator

image_colors = ImageColorGenerator(mask)

In [ ]:
# 이미지로 마스킹한 워드크라우드 만들기

wordcloud2 = WordCloud(font_path=path,
                     relative_scaling=0.2, mask=mask,
                     background_color='white').generate_from_frequencies(dict(data))

plt.figure(figsize=(12,12))
plt.imshow(wordcloud2.recolor(color_func=image_colors), interpolation='bilinear')
plt.axis('off')
plt.show()